# Sifaka Mapping 

### Imports

In [1]:
import pandas as pd
import requests

### Load Excel File

In [2]:
excel_location = "data/sifaka_locations.xlsx"
xls = pd.ExcelFile(excel_location)
df = xls.parse(xls.sheet_names[0])
# Show that the data frame loaded
print (df.head())

  Sample ID  Species Name Sex  Location        Latitude        Longitude
0    MOR101  P. verreauxi   F  Beroboka  S24Â°45'53.0''  E046Â°51'45.0''
1     ZAH26    P. diadema   M  Zahamena  S24Â°45'45.7''  E046Â°51'48.7''


### Convert DMS to DD

In [3]:
conversion_server = 'http://ice:8081/'
split_set = set(['s', 'a', 'e', "'", 
                     '"', '`', ":", '\xcb\x9a', 
                     '\xc2\xb0', '\xc2\xba', 'â' 
                     '°', 'º', '˚', ',', ' '])



def process_lat_lon(dms):
    dms = [dms.lower()]
    new_items = []
    for num, i in enumerate(split_set):
        new_items = [j.split(i) for j in dms if i]
        items_replacement = []
        for i in new_items:
            for j in i:
                if j:
                    items_replacement.append(j)
        dms = items_replacement
    try:
        items = list(map(float, dms))
        keys = ['d', 'm', 's']
        return_obj = {}
        for num, item in enumerate(items):
            return_obj[keys[num]] = item
        for key in keys:
            if key not in return_obj:
                return_obj[key] = 0
        return return_obj
    except:
        print(dms)
        return False

def run_conversions():
    for num, lat_lon in enumerate(zip(df['Latitude'], df['Longitude'])):
        lat, lon = lat_lon
        proc_lats = process_lat_lon(lat)
        proc_lons = process_lat_lon(lon)
        conversion_str = "?dlat={}&mlat={}&slat={}&dlon={}&mlon={}&slon={}".format(-proc_lats['d'], 
                                                                                    proc_lats['m'], 
                                                                                    proc_lats['s'], 
                                                                                    proc_lons['d'], 
                                                                                    proc_lons['m'], 
                                                                                    proc_lons['s'])
        request_str = conversion_server + conversion_str
        response = requests.get(request_str).json()
        df["Latitude"][num], df["Longitude"][num] = str(response['latitude']), str(response['longitude'])

run_conversions()

### Write to a CSV file

In [4]:
df.to_csv('output/sifaka_locations_converted_dd.csv', sep=',')